# PROJET Data Engineering
Les objectifs de ce projet sont les suivants:

> *   Récuperer tous les départs et arrivées des avions de tout les aéroports sur le site //EXEMPLE//.
*   Récuperer la météo correspondant aux aéroports.
*   Correler les données dans un but statistique pour évaluer l'influence de la météo et des companie sur les retards.
*   Afficher les aéroports et les avions sur une carte 
*   Pourquoi pas voir le taux de remplissage des avions ([Site interressant](https://www.vol-retarde.fr/blog/2018/03/01/quel-est-le-taux-de-remplissage-des-compagnies-aeriennes)) 





## Imports

In [1]:
import time
import sys
import re
from time import sleep
from metar_taf_parser.parser.parser import MetarParser
import bs4
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import seaborn as sb
import folium as fl
import lxml
from folium.plugins import MarkerCluster
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--dns-prefetch-disable')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


/tmp/ipykernel_221133/1162379615.py:23: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


## Récupération des vols

#### Création des fonctions de récupération des vols





In [2]:
def RecupVol (dep,airport) : 
    """
    dep = departures ; !dep = arrivals ; airport indique l'aéroport demandé
    """
    currBrowser = browser
    if dep :
        url = "//EXEMPLE//"+airport+"/departures"
    else :
        url = "//EXEMPLE//"+airport+"/arrivals"
    #si on a une erreur on reset le driver et on recommence(necessaire lorsqu'on se fait ban)
    while(True):
        currBrowser.get(url)
        sleep(1)
        try:
            element = currBrowser.find_element(By.XPATH,"//button[contains( text( ), 'Load earlier flights')]")
        except:
            currBrowser.delete_all_cookies()
            continue;
        break;
        
    for i in range(2):
        sleep(1) #attente du chargement des vols
        currBrowser.execute_script("arguments[0].click();", element)
    page_source = browser.page_source
    return page_source,dep

In [11]:
global listDate, listDateCopy, listDateCopy2
 

def inDF (ps) :
    soup = BeautifulSoup(ps[0], 'html.parser')
    table = soup.find_all('table')
    
    global listDate, listDateCopy,listDateCopy2
    listDate = soup.find_all("tr", {"class": "row-date-separator hidden-xs hidden-sm"})#récupération des dates
    listDateCopy = listDate.copy()
    listDateCopy2 = listDate.copy()

    df = pd.read_html(str(table))[0]
    df.drop(df.tail(2).index,inplace=True) # Drop des 2 premières et 
    df.drop(df.head(3).index,inplace=True) # dernière lignes car incorrecte


    tsize = int(df.shape[0]/2)                  # Les valeurs sont doublés au moment de la récupération donc on drop 
    df.drop(df.head(tsize).index,inplace=True)  # la première moitié du DataFrame


    if ps[1] :
        df.set_axis(['Time', 'Flight', 'To', 'Airline', 'Aircraft','Block','Status'], axis=1, inplace=True) # Réarrangement des noms de colonnes et drop d'une colonne en trop
        df['To'] = df.apply(lambda x : re.sub(r'-', '', x['To'] ),axis=1)
    else :
        df.set_axis(['Time', 'Flight', 'From', 'Airline', 'Aircraft','Block','Status'], axis=1, inplace=True) # La colonne BLock correspond à la couleur du vol sur le site Vert = à l'heure
        df['From'] = df.apply(lambda x : re.sub(r'-', '', x['From'] ),axis=1)
  
    #on applique à toute les ligne la fonciton addColumSec qui va calculer 
    #le nombre de secondes écoulées entre l'"epoch time" et la date d'arrivée prévue
    df['SecTimeStamp'] = df.apply(lambda x :addColumSec(x),axis=1)
    df['FullDate'] = df.apply(lambda x : addColumFullDate(x),axis=1)

    df = df.rename(index=lambda s : s - tsize - 1 ) # On renomme les lignes 

    df["secRetard"] = df.apply(lambda x : schToSecRetard(x),axis=1)
    df['Airline'] = df.apply(lambda x : re.sub(r' -', '', x['Airline'] ),axis=1)

    df = df[df["Status"] != "NaN"]
    df = df[df["secRetard"] != "NaN"]
    df.drop(["SecTimeStamp","Block"],inplace=True,axis=1)
    return  df

def addColumSec(x):
    global listDate
    try:
        #on commence par récupérer le nombre de secondes écoulées dans la journée
        dayTab =x.iloc[0].split(":") 
        print(str(dayTab))
        SecDay = float(dayTab[0])*3600+float(dayTab[1])*60
        
        #puis le nombre de secondes écoulées dans l'année
        tmpDate = listDate[0].getText()
        Date = time.strptime("2021 "+ tmpDate,"%Y %A, %b %d")
        SecDate = time.mktime(Date)
        return SecDay + SecDate
    except (ValueError, IndexError):
        listDate.pop(0)


def addColumFullDate(Li):
    """
    Fonction servant à ajouter la date complete qui servira après a merge la df de la meteo et celle des avions
    """
    global listDateCopy2
    #transformer les string en données exploitables
    if((Li.iloc[6]).startswith("Departed ") | (Li.iloc[6]).startswith("Landed ")):
        Li.iloc[6]= Li.iloc[6][-5:]
    if( (Li.iloc[6]).startswith("Scheduled") | (Li.iloc[6]).startswith("Estimated") | (Li.iloc[6]).startswith("Delayed")  | (Li.iloc[6]).startswith("Unknown") | (Li.iloc[6]).startswith("Departed")):
        return "NaN"
    if((Li.iloc[6]).startswith("Canceled")):
        return "Canceled"


    try:
        #on récupère la date pour pouvoir plus facilement la mettre en forme dans la colonne et si on n'arrive pas à la recupérer cela veut dire que l'on est
        #tombés sur une ligne avec la date
        T = time.strptime(Li.iloc[6],'%H:%M')

        tmpDate = listDateCopy2[0].getText()
        Date = time.strptime("2021 "+ tmpDate,"%Y %A, %b %d")

    # on met les minute à la valeur du dessous (possible amélioration en rapprochant à la valeur la plus proche mais implique de prendre en compre les changement d'heure, de jours, de mois etc)
        if(T.tm_min <30):
            mins = "00"
        else:
            mins = "30"
        
        DayNum = str(Date.tm_mday)
        if (len(DayNum)==1):
            DayNum = "0"+DayNum
        return str(Date.tm_year) + "-" + str(Date.tm_mon) + "-" + DayNum + " " + str(T.tm_hour) + ":" + mins + ":" + "00"

    except ValueError:
        listDateCopy2.pop(0)
    



def schToSecRetard(Li):
    """
    fonction modifiant la colonne du statut pour calculer le retard:
    """
    global listDateCopy
    #ETAPE 1 :transformer les string en données exploitables(optimisable répétition de code)
    if((Li.iloc[6]).startswith("Departed ") | (Li.iloc[6]).startswith("Landed ")):
        Li.iloc[6]= Li.iloc[6][-5:]
    if( (Li.iloc[6]).startswith("Scheduled") | (Li.iloc[6]).startswith("Estimated") | (Li.iloc[6]).startswith("Delayed")  | (Li.iloc[6]).startswith("Unknown") | (Li.iloc[6]).startswith("Departed")):
        return "NaN"
    if((Li.iloc[6]).startswith("Canceled")):
        return "Canceled"

    #ETAPE 2 : Calculer le nombre de secondes total entre l'an 0 et l'arrivée ou le départ
    try: 
    #on commence par récupérer le nombre de secondes écoulées dans la journée
        dayTab =Li.iloc[6].split(":") 
        SecDay = float(dayTab[0])*3600+float(dayTab[1])*60
        
        #puis le nombre de secondes écoulées dans l'année
        tmpDate = listDateCopy[0].getText()
        Date = time.strptime("2021 "+ tmpDate,"%Y %A, %b %d")
        SecDate = time.mktime(Date)
        
        #print("day: "+str(SecDay)+"  date: "+str(SecDate)+"   iloc7: "+str(Li.iloc[7]))
        return (SecDay + SecDate)-Li.iloc[7]
    except (ValueError, IndexError):
        listDateCopy.pop(0)


## Récupération de la Météo


#### **V2** - Fonction récupération et traitement du code METAR

In [4]:

## Fonction de récupération avec le code METAR et la librairy metar-taf-parser
#cette fonction nous permet plus de fiabilité et de données que la fonction précédente
#lien librairy https://github.com/mivek/python-metar-taf-parser




def RecupMetarAirport(Name):
    urlWeather = "//EXEMPLE//"+Name+"/weather"

    hdr = {'User-Agent': 'Mozilla/5.0'}
    Req = urllib.request.Request(urlWeather,headers=hdr)
    html = urllib.request.urlopen(Req)


    WeatherSoup = BeautifulSoup(html,'html.parser')
    WeatherTable = WeatherSoup.find_all("table")
   
    dfW = pd.read_html(str(WeatherTable))[0]

#récupération des dates dans un dataframe
    dfWDate = dfW["UTC DATE/TIME"]  
    dfWDate = dfWDate.iloc[::2]   # une ligne sur deux n'est pas utile pour nous, on va donc les drop
    dfWDate = dfWDate.reset_index(drop=True)  #reset des index des lignes
    

  #on récupère le code METAR
    dfWMetar = dfW["METAR"]  
    dfWMetar = dfWMetar.iloc[::2]
    dfWMetar = dfWMetar.reset_index(drop=True)  #reset des index
  

  #mise en forme dans une dataframe
    dfWMetar=pd.DataFrame(dfWMetar)

    dfWMetar["DateTime UTC"] = dfWDate.apply(lambda x :ajustTime(x))



  #ajout de toutes les valeurs dans les colonnes ajoutées
    return dfWMetar

def ajustTime(x):
    if(float(x[-5:-3]) <30):
        x=x[:-5] + "00" + x[-3:]
    else:
        x=x[:-5] + "30" + x[-3:]
    return x
    

## Récupération des codes IATA et des informations sur les aéroports





#### **V3** - Recupération à partir d'un meilleur fichier csv

In [5]:
def RecupLargeAirport():
    df = pd.read_csv("https://raw.githubusercontent.com/datasets/airport-codes/master/data/airport-codes.csv")
    df = df[df['type'] == 'large_airport']
    df.drop(["ident","type","elevation_ft","continent","name","iso_region","municipality","gps_code","local_code","coordinates"],inplace=True,axis=1)
    return df

## Utilisation des données

>Nous avons maintenant ces fonctions : 
>
> Pour la récupération des vols :
>
> Pour avoir les départs : dep = true et pour les arrivées : dep = false
 ```
inDF(RecupVol(dep,airport)) 
```
>Pour la récupération de la méteo :
 ```
RecupMetarAirport(airport)
```
>Pour la récupération des codes IATA à partir de wikipedia(full useless):
 ```
RecupAllIATA()
```
>V2 de la récupération des IATA et d'autres données sur les aéroports(à peaufiner?)
 ```
RecupAirportFromCSV()
```

>[lien ancien TP de DE pour l'utilisation des librairies et statistiques](https://colab.research.google.com/drive/1htRGO4mk6XT2fUqBE8rHTOJpqOUT9PBC#scrollTo=uDzdmRdbk7fM)


### Mise en forme et exploitation des données des avions et de la météo

#### Récupération et mise en forme

##### Récupération de la météo et de la liste des départ d'avion de tout les aéroports de france
>*on met en moyenne 8-10 secondes pour un aéroport donc pour les 158 de france(que l'on a) on mettra à peut près 30mins*

In [9]:
def RecupCountryPlaneWeather(maxValue = 10000,departure =True,SaveWhileRunning=False):
    """
    Fonction de récupération de tous les aéroports d'un pays
    Param country : le pays concerné si pas défini alors on va récuperer ceux de france (FR)
    Param departur : booleen true pour sélectionner les départs ou les arrivées
    Param maxValue : le nombre d'aéroports maximum que l'on veut récuperer avec notre requete
    Param SaveWhileRunning : bolleen permettant de spécifier si on append les résultats pendant la récupération (Préférable pour éviter la perte de données lors d'une erreur ou d'un ban)
    """
     # Récupération des codes IATA correspondant au pays
    DFIATA = RecupLargeAirport()
    DFIATA=DFIATA.dropna().reset_index(drop=True)
    SerieIATA = SerieIATA=DFIATA["iata_code"].str.lower()

    
  #Récupération de la méteo et des avions des aéroports 
    DFallPlaneWeather = pd.DataFrame()
    i=0
    for row in SerieIATA:
        print(i)
        print(row)
        if (i >= maxValue):
            break
        i=i+1;
    #Récupération
        #try:
        DFM = RecupMetarAirport(str(row))
        sleep(0.5)
        DFAirport = inDF(RecupVol(True,str(row)))
        #except :
        #    print("erreur pour l'aéroport "+row)
        #    continue
        #merge des dataframes
        DFCurrentPLaneWeather = pd.merge(DFM,DFAirport,how='inner',left_on='DateTime UTC', right_on='FullDate')
        #ajout de leurs codes iata
        DFCurrentPLaneWeather["iata"] = row
        #print(DFCurrentPLaneWeather)
        #ajout dans le csv
        if(SaveWhileRunning):
            DFCurrentPLaneWeather.to_csv('allDataMieux.csv',mode='a')
        #ajout dans la dataframe complete
        try:
            if(i==1):
                DFallPlaneWeather=DFCurrentPLaneWeather
            else:
                DFallPlaneWeather = pd.concat([DFallPlaneWeather,DFCurrentPLaneWeather])
      #print(DFallPlaneWeather)
        except:
            continue
 
    return DFallPlaneWeather
  



### Code à executer régulièrement


In [12]:
DFFR = RecupCountryPlaneWeather(SaveWhileRunning=True,maxValue=2)# à peu près 45 minutes d'execution

0
pom
["{{objFlight.flight.time.scheduled.departure * 1000 || '-' | date", ' timeFormat ', ' timeZone}}']


IndexError: pop from empty list